In [38]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

%matplotlib inline

In [39]:
use_sample= True
random_state = 1024

In [40]:
path_data = Path('data/vehicles.feather')
if not path_data.exists():
    df_cars = pd.read_csv("data/vehicles.csv")
#     save as .feather for future. Faster.
    df_cars.to_feather(path_data)
else :
    df_cars = pd.read_feather(path_data)

Check for missing values

In [41]:
df_cars.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

Since there are a lot of missing values, we will use the columns with few missing values

In [42]:
cols = ['id','year','fuel','odometer','transmission', 'price']
df_cars = df_cars[cols]

In [43]:
df_cars = df_cars.loc[~df_cars.isna().any(axis = 1)]

In [44]:
if use_sample:
    df = df_cars.sample(frac = .1, random_state=random_state)
else:
    df = df_cars.copy()

Ensure that the ids are unique in the dataset.

If not, select first instance of each id.

In [45]:
try:
    assert len(df.id.unique()) == df.shape[0]
except:
    print("there are multiple rows for the same ID")
    print("selecting the first row for each ID")
    df.groupby(['id']).first().reset_index(drop =False,inplace=True)

convert all string dtypes to categorical

In [46]:
# from pandas.api.types import is_string_dtype
# for col in df.columns:
#     if is_string_dtype(df[col]):
#         df[col] = pd.Categorical(df[col])
# df.dtypes

Split to train and test

In [47]:
categorical = ['fuel','transmission']
numeric = ['year','odometer']
target = 'price'

In [48]:
df[categorical] = df[categorical].astype(str)

Split to train, validation and test datasets.

- Train dataset       - old cars (cars with year < 2017) => 74%
- Validation dataset  - newer cars (2017 <= year <=2018) => 15%
- Test dataset        - newest cars (2019 <= year)       => 11%

In [49]:
df.head()

,id,year,fuel,odometer,transmission,price
411308,7311450486,2013.0,gas,46267.0,automatic,15995
340463,7303076328,2020.0,gas,21938.0,other,25990
311669,7314370012,1998.0,gas,320220.0,automatic,123
262500,7306571989,2017.0,diesel,29014.0,other,19590
33923,7316374412,2005.0,gas,130000.0,automatic,1500


In [50]:
(pd.DataFrame(df.year.value_counts(normalize=True))
 .reset_index(drop = False)
 .sort_values(by = 'year', ascending = False)
 .head(10))

,year,proportion
59,2022.0,0.000431
22,2021.0,0.005724
9,2020.0,0.044857
6,2019.0,0.060615
0,2018.0,0.088205
1,2017.0,0.083990
3,2016.0,0.071776
2,2015.0,0.074267
5,2014.0,0.069333
4,2013.0,0.071081


In [54]:
dftr = df.loc[df.year < 2017].copy()
dfval = df.loc[df.year.isin([2017, 2018])].copy()
dfte = df.loc[df.year > 2018].copy()

Fit DictVectorizer for categorical columns

In [55]:
dv = DictVectorizer()
tr_dicts = dftr[numeric + categorical].to_dict(orient = 'records')
xtr = dv.fit_transform(tr_dicts)
ytr = dftr[target].values
val_dicts = dfval[numeric + categorical].to_dict(orient='records')
xval = dv.transform(val_dicts)
yval = dfval[target].values
te_dicts = dfte[numeric+categorical].to_dict(orient = 'records')
xte = dv.transform(te_dicts)
xte = dfte[target].values

Train Models

In [128]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb